In [1]:
import sympy as sp

# Define coordinates
u,v = sp.symbols('u v', real =True)

#Fist FF
def first_fundamental_form(X):
    Xu = X.diff(u)
    Xv = X.diff(v)
    
    E = sp.simplify(Xu.dot(Xu)) 
    F = sp.simplify(Xu.dot(Xv))
    G = sp.simplify(Xv.dot(Xv))

    I = sp.Matrix([[E, F], [F, G]])
    return I


# 2nd FF
def second_fundamental_form(X, u, v):
    # Calculate partial derivatives
    Xu = X.diff(u)
    Xv = X.diff(v)
    Xuu = X.diff(u, 2)
    Xuv = X.diff(u, v)
    Xvv = X.diff(v, 2)

    # Compute the normal vector
    N = Xu.cross(Xv)
    N = sp.simplify(N)
    N_norm = sp.trigsimp(N.norm())
   
    # Manually simplify N_norm if possible
    # For the sphere, N_norm simplifies to sin(u)**2
    N_norm_simplified = sp.simplify(N_norm)

    # Compute the unit normal vector
    N_vector = N / N_norm_simplified
    N_vector = sp.simplify(N_vector)
    # Calculate the coefficients of the second fundamental form and simplify
    L = sp.trigsimp(N_vector.dot(Xuu))
    M = sp.trigsimp(N_vector.dot(Xuv))
    N_coeff = sp.trigsimp(N_vector.dot(Xvv))

    # Construct and simplify the second fundamental form matrix
    II = sp.Matrix([[L, M], [M, N_coeff]])

    # Apply trigonometric simplification to the entire matrix
    II = II.applyfunc(sp.trigsimp)

    return II


def gaussian_curvature(X):
    I = first_fundamental_form(X)
    II = second_fundamental_form(X,u,v)
    
    E = I[0, 0]
    F = I[0, 1]
    G = I[1, 1]
    L = II[0, 0]
    M = II[0, 1]
    N = II[1, 1]
    
    K = (L*N - M**2) / (E*G - F**2)
    K = sp.simplify(K)
    return K

def mean_curvature(X):
    I = first_fundamental_form(X)
    II = second_fundamental_form(X,u,v)
    
    E = I[0, 0]
    F = I[0, 1]
    G = I[1, 1]
    L = II[0, 0]
    M = II[0, 1]
    N = II[1, 1]
    
    H = (E*N + G*L - 2*F*M) / (2*(E*G - F**2))
    H = sp.simplify(H)
    return H

# Function to calculate Christoffel symbols for any metric
def christoffel_symbols(metric, coords):
    n = len(coords)  # Dimension
    
    # inverse of the metric
    g_inv = metric.inv()

    # Initialize the Christoffel symbols as a 3D tensor (n x n x n)
    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)]

    # Compute the Christoffel symbols
    for i in range(n):
        for j in range(n):
            for k in range(n):
                # Christoffel symbol \Gamma^k_{ij}
                Gamma[i][j][k] = 0
                for l in range(n):
                    Gamma[i][j][k] += 1/2 * g_inv[k, l] * (
                        sp.diff(metric[l, j], coords[i]) +
                        sp.diff(metric[l, i], coords[j]) -
                        sp.diff(metric[j, i], coords[l])
                    )
                Gamma[i][j][k] = sp.simplify(Gamma[i][j][k])
    
    return Gamma

# Parametrization of a sphere in standard spherical coordinates
x = sp.sin(u) * sp.cos(v)
y = sp.sin(u) * sp.sin(v)
z = sp.cos(u)
X = sp.Matrix([x, y, z])

# Compute the first fundamental form
I = first_fundamental_form(X)
print("First Fundamental Form (I):")
print(I)
print("\n")

# Compute the second fundamental form
II = second_fundamental_form(X,u,v)
print("Second Fundamental Form (II):")
print(II)
print("\n")

# Compute the Christoffel symbols
coords = [u,v]
Gamma = christoffel_symbols(I, coords)

# Display the Christoffel symbols
print("Christoffel Symbols:")
for i in range(2):
    for j in range(2):
        for k in range(2):
            gamma = Gamma[i][j][k]
            if gamma != 0:
                symbol_name = f"Γ^{coords[k]}_{{{coords[i]}{coords[j]}}}"
                print(f"{symbol_name} = {gamma}")
print("\n")

#Print Gaussian Curvature
K = gaussian_curvature(X)
print("Gaussian Curvature (K):")
print(K)
print("\n")

#Print Mean Curvature
H = mean_curvature(X)
print("Mean Curvature (H):" )
print(H)



First Fundamental Form (I):
Matrix([[1, 0], [0, sin(u)**2]])


Second Fundamental Form (II):
Matrix([[-sin(u)/Abs(sin(u)), 0], [0, -sin(u)**3/Abs(sin(u))]])


Christoffel Symbols:
Γ^v_{uv} = 1.0/tan(u)
Γ^v_{vu} = 1.0/tan(u)
Γ^u_{vv} = -0.5*sin(2*u)


Gaussian Curvature (K):
1


Mean Curvature (H):
-sin(u)/Abs(sin(u))
